In [10]:
!java -version

openjdk version "1.8.0_362-362"
OpenJDK Runtime Environment (build 1.8.0_362-362-b09)
OpenJDK 64-Bit Server VM (build 25.362-b09, mixed mode)


# **IRIS Naive Bayes Classification with Pyspark**

In [11]:
# install
# !pip install Pyspark
!pip install pyspark==3.3.1

# Conexión con el cluster de Spark con pyspark

In [17]:
# import os
# os.environ["AWS_REGION"]
# os.environ["AWS_ACCESS_KEY_ID"]

In [18]:
from pyspark.sql import SparkSession

#os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.iceberg:iceberg-spark-runtime-3.2_2.12:1.1.0,org.mariadb.jdbc:mariadb-java-client:2.3.0,org.postgresql:postgresql:42.5.4 pyspark-shell'
dst_lh_url  = os.environ.get('DST_LH_URL', 'spark://spark-master-0.spark-headless.demml.svc.cluster.local:7077')
dst_lh_appn = os.environ.get('DST_LH_APPN', 'iris_naive_bayes_classification_pyspark_EDU')
import subprocess
my_pod_ip = subprocess.run(['hostname', '-I'], stdout=subprocess.PIPE).stdout.decode('utf-8').strip(' \n\t')
my_pod_ip

spark = SparkSession.builder \
    .master(dst_lh_url) \
    .appName(dst_lh_appn) \
    .config('spark.jars.packages','org.apache.iceberg:iceberg-spark-runtime-3.3_2.12:1.1.0,org.mariadb.jdbc:mariadb-java-client:2.3.0,org.postgresql:postgresql:42.5.4,software.amazon.awssdk:bundle:2.17.257,software.amazon.awssdk:url-connection-client:2.17.257,software.amazon.awssdk:s3:2.17.257,software.amazon.awssdk:iam:2.17.257,org.apache.hadoop:hadoop-aws:3.2.3') \
    .config('spark.sql.extensions','org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions') \
    .config('spark.sql.catalog.spark_catalog','org.apache.iceberg.spark.SparkSessionCatalog') \
    .config('spark.sql.catalog.spark_catalog.catalog-impl','org.apache.iceberg.jdbc.JdbcCatalog') \
    .config('spark.sql.catalog.spark_catalog.uri','jdbc:postgresql://postgresql:5432/iceberg') \
    .config('spark.driver.host', my_pod_ip) \
    .config('spark.sql.catalog.spark_catalog.jdbc.useSSL','false') \
    .config('spark.sql.catalog.spark_catalog.jdbc.user','iceberg') \
    .config('spark.sql.catalog.spark_catalog.jdbc.password','iceberg') \
    .config('spark.sql.catalog.spark_catalog.warehouse','s3a://warehouse/') \
    .config('spark.sql.catalog.spark_catalog.s3.endpoint','http://minio:9000') \
    .config('spark.sql.catalog.spark_catalog.io-impl', 'org.apache.iceberg.aws.s3.S3FileIO') \
    .config('spark.sql.defaultCatalog','spark_catalog') \
    .config('spark.hadoop.fs.s3a.endpoint','http://minio:9000') \
    .config('spark.hadoop.fs.s3a.access.key','admin') \
    .config('spark.hadoop.fs.s3a.secret.key','t4bl4red0nd4') \
    .config('spark.hadoop.fs.s3a.path.style.access','true') \
    .config('spark.hadoop.fs.s3a.impl','org.apache.hadoop.fs.s3a.S3AFileSystem') \
    .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false") \
    .config("spark.hadoop.fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider") \
    .getOrCreate()

import datetime as dt
ini = dt.datetime.now()

spark

In [19]:
# read sklearn inbuilt data
import pandas as pd
import numpy as np
from sklearn.datasets import load_iris
iris = load_iris(as_frame=True)
iris = iris.frame
iris = spark.createDataFrame(iris)

AttributeError: 'DataFrame' object has no attribute 'iteritems'

In [ ]:
iris.show()

In [ ]:
iris.printSchema()

In [ ]:
iris.columns

In [ ]:
from pyspark.ml.feature import VectorAssembler 

In [ ]:
featureassembler= VectorAssembler(inputCols=['sepal length (cm)',
 'sepal width (cm)',
 'petal length (cm)',
 'petal width (cm)'], outputCol='Features')

In [ ]:
output = featureassembler.transform(iris)

In [ ]:
output.show()

In [ ]:
modeldata = output.select('Features','target')

In [ ]:
modeldata.show()

In [ ]:
# split data
train_data, test_data = modeldata.randomSplit([0.8,0.2])

In [ ]:
train_data.show()

In [ ]:
# naive bayes model
from pyspark.ml.classification import NaiveBayes

In [ ]:
nb = NaiveBayes(featuresCol='Features', labelCol='target')

In [ ]:
nb = nb.fit(train_data)

In [ ]:
# prediction
y_pred = nb.transform(test_data)

In [ ]:
y_pred.show()

In [ ]:
# confusion matrix
y_pred.groupBy('target', 'prediction').count().show()

In [ ]:
from sklearn.metrics import confusion_matrix
pred = y_pred.select("prediction").collect()
orig = y_pred.select("target").collect()
print(confusion_matrix(orig, pred))

In [ ]:
# evaluation
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [ ]:
evaluator = MulticlassClassificationEvaluator(
    labelCol='target', predictionCol='prediction'
)

In [ ]:
evaluator.metricName

In [ ]:
accuracy = evaluator.evaluate(y_pred)
precision = evaluator.evaluate(y_pred, {evaluator.metricName: 'weightedPrecision'})
recall = evaluator.evaluate(y_pred, {evaluator.metricName: 'weightedRecall'})
f1_score = evaluator.evaluate(y_pred, {evaluator.metricName: 'f1'})

print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1_score}')

In [ ]:
# close connection to spark
spark.stop()

In [ ]:
fin = dt.datetime.now()

print(str(fin - ini))